# Teoria


In [ ]:
#https://github.com/Awantikashri/Gravitational-Wave-Detection-Using-Deep-Learning/blob/main/g2net-eda-preprocessing-model.ipynb
# Como el mio pero con cositas, mirarlo


# EDA

In [ ]:
#https://github.com/YuriMiyamori/g2net-gravitational-wave-detection/blob/main/notebooks/00_EDA.ipynb

using DataFrames, DataFramesMeta, CSV
using NPZ
using Random, Statistics
using DSP,FFTW, AbstractFFTs,Interpolations
using Wavelets
using ContinuousWavelets

using PyPlot
plt.style.use("seaborn-whitegrid");
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 12
rcParams["figure.figsize"] = [16,16]
rcParams["figure.dpi"] = 220;



h = data[:, 1]
h_in_sig = data_in_sig[:, 1]
h̃ = dt * FFTW.rfft(h) 
h̃_in_sig = dt * FFTW.rfft(h_in_sig) 




function noise_detect(h, h̃, tnumber_of_chunks, overlap_rate)
    t = (1:length(h)) .*dt  |> collect
    frequencies = FFTW.rfftfreq(length(h), sampling_rate)
    # frequencies = FFTW.rfftfreq(length(h), sampling_rate)
    points_per_chunk = 2^floor(Int, log2(length(h)/number_of_chunks))

    s = welch_pgram(h, points_per_chunk, floor(Int, points_per_chunk*overlap_rate), fs=sampling_rate, window=hanning)
    f_noise, noise_welch = s.freq, s.power

    noise_spectral_density = sqrt.(2 * length(h) .* noise_welch ./ sampling_rate)
    nodes = (f_noise,)
    itp = interpolate(nodes, noise_spectral_density, Gridded(Linear()))
    noise_spectral_density_interpolator = itp(frequencies);
    @show(size(noise_spectral_density_interpolator))
    fig, ax = plt.subplots(figsize=(16,8))
    ax.loglog(frequencies, abs.(h̃), label="Raw data", alpha=0.3)
    ax.loglog(f_noise, noise_spectral_density, label="Noise estimate")
    ax.set_xlim(8, 0.6*sampling_rate)
    ax.set_xlabel("Frequency (Hz)")
    ax.set_ylabel("Noise spectrum and strain Fourier transform (seconds)")
    ax.legend();
    # @show(htilde ./ noise_spectral_density_interpolator)

    h̃_equalized = h̃ ./ noise_spectral_density_interpolator
    #h_equalized = fade(sampling_rate * FFTW.irfft(htilde_equalized, length(t)));
    h_equalized = (sampling_rate * FFTW.irfft(h̃_equalized, length(t)));
    score = mean(abs.(h_equalized))
    ax.set_title("number_of_chunks:: $number_of_chunks, overlap_rate:: $overlap_rate => score:: $score")

    # Now we transform back to the time domain, and smoothly fade on and off to get rid of loud clicks
    plot_td_and_fd(h_equalized,  h̃ ./ h̃[length(h̃) - div(length(h̃), 4)]/10, h̃_equalized)
    return h_equalized, h̃_equalized
end

number_of_chunks = 16
h_equalized, h̃_equalized= noise_detect(h, h̃, number_of_chunks, 0.5)
h_equalized_in_sig, h̃_equalized_in_sig= noise_detect(h_in_sig, h̃_in_sig, number_of_chunks, 0.5)

"""
for number_of_chunks in 2:8
    noise_detect(number_of_chunks, 0.8)
end
"""

# Preprocessing

In [ ]:
#https://github.com/YuriMiyamori/g2net-gravitational-wave-detection/blob/main/notebooks/LOSC_Event_tutorial.ipynb
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    freqs1 = np.linspace(0,2048.,Nt/2+1)

    # whitening: transform to freq domain, divide by asd, then transform back, 
    # taking care to get normalization right.
    hf = np.fft.rfft(strain)
    norm = 1./np.sqrt(1./(dt*2))
    white_hf = hf / np.sqrt(interp_psd(freqs)) * norm
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

# make wav (sound) files from the whitened data, +-2s around the event.

from scipy.io import wavfile

# function to keep the data within integer limits, and write to wavfile:
def write_wavfile(filename,fs,data):
    d = np.int16(data/np.max(np.abs(data)) * 32767 * 0.9)
    wavfile.write(filename,int(fs), d)

deltat_sound = 2.                     # seconds around the event

# index into the strain time series for this time interval:
indxd = np.where((time >= tevent-deltat_sound) & (time < tevent+deltat_sound))

# write the files:
write_wavfile(eventname+"_H1_whitenbp.wav",int(fs), strain_H1_whitenbp[indxd])
write_wavfile(eventname+"_L1_whitenbp.wav",int(fs), strain_L1_whitenbp[indxd])

# re-whiten the template using the smoothed PSD; it sounds better!
template_p_smooth = whiten(template_p,psd_smooth,dt)

# and the template, sooming in on [-3,+1] seconds around the merger:
indxt = np.where((time >= (time[0]+template_offset-deltat_sound)) & (time < (time[0]+template_offset+deltat_sound)))
write_wavfile(eventname+"_template_whiten.wav",int(fs), template_p_smooth[indxt])


from IPython.display import Audio

fna = eventname+"_template_whiten.wav"
print(fna)
Audio(fna)


fna = eventname+"_H1_whitenbp.wav"
print(fna)
Audio(fna)




# Model

In [ ]:
#https://github.com/JamesMcGuigan/kaggle-gravitational-waves/blob/master/src/experiments/rnn_starter/model.py
import tensorflow as tf
from tensorflow.keras import layers


def build_model():
    inputs = layers.Input(shape=(27, 128))

    gru1  = layers.Bidirectional(layers.GRU(128, return_sequences=True), name='gru_1')
    gru2  = layers.Bidirectional(layers.GRU(128, return_sequences=True), name='gru_2')
    pool1 = layers.GlobalAveragePooling1D(name='avg_pool')
    pool2 = layers.GlobalMaxPooling1D(name='max_pool')

    x = gru1(inputs)
    x = gru2(x)
    x = tf.keras.layers.Concatenate()([pool1(x), pool2(x)])

    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(1,   activation="sigmoid", name="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

In [ ]:
#https://github.com/pranay1990/G2NET_Gravitational_Wave_Detection/blob/main/effnetb6-cwt.ipynb

def get_model(input_shape=input_shape):
    model = Sequential()
    model.add(Conv2D(16,input_shape=input_shape, kernel_size=3,))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2))
    model.add(Activation("relu"))
    model.add(Conv2D(32,kernel_size=1,dilation_rate=1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2)))
    model.add(Activation("relu"))
    model.add(Conv2D(64,kernel_size=1 ,dilation_rate=1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2))
    model.add(Activation("relu"))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer = Adam(lr=1e-3),loss='binary_crossentropy',metrics=['AUC'])
    return model

model = get_model(input_shape)
model.summary()


lr_reducer = ReduceLROnPlateau(monitor='val_loss',factor=np.sqrt(0.01), cooldown=0,\
                               patience=3, min_lr=0.5e-8,mode='min')
# This file path where the best weights will be saved
checkpoint_filepath = '/content/tfm_g2n/model_cnn.h5'
# It saves the weights which shows the highest validation accuracy
modelchck=ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,verbose=1,
                          monitor='val_auc',mode='max',save_best_only=True)

model.fit(train_dataset, validation_data=valid_dataset, epochs = 10, bach_size= 250, callbacks=[lr_reducer,modelchck])